# GIS Wind-Methane Dispersion Analysis - Spatial Interpolation

This notebook focuses on spatial interpolation techniques to estimate methane concentrations across the study area. We'll implement and compare several interpolation methods to create continuous surfaces from our discrete sensor data.

## Objectives
1. Implement multiple spatial interpolation methods (IDW, RBF, Kriging)
2. Compare and evaluate the performance of different methods
3. Create visual maps of methane concentration distribution
4. Analyze how wind affects interpolation results

## 1. Setup and Data Loading

In [ ]:
# Import libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from shapely.geometry import Point
import folium
from scipy.interpolate import griddata, Rbf
from pykrige.ok import OrdinaryKriging
import contextily as ctx
from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable

# Set up plotting parameters
plt.style.use('seaborn-whitegrid')
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 10)
plt.rcParams['font.size'] = 12
sns.set_style("whitegrid")

In [ ]:
# Import from our project modules
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from src.data_processing import load_data, preprocess_methane_data, preprocess_wind_data
from src.interpolation import create_grid, idw_interpolation, rbf_interpolation, kriging_interpolation, plot_interpolation

In [ ]:
# Define file paths
project_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
methane_path = os.path.join(project_dir, 'data', 'methane_sensors.csv')
wind_path = os.path.join(project_dir, 'data', 'wind_data.csv')

# Alternative direct paths if needed
if not os.path.exists(methane_path):
    methane_path = r"C:\Users\pradeep dubey\Downloads\methane_sensors.csv"
    wind_path = r"C:\Users\pradeep dubey\Downloads\wind_data.csv"

# Load data
methane_df, wind_df = load_data(methane_path, wind_path)
methane_gdf = preprocess_methane_data(methane_df)
wind_df_processed = preprocess_wind_data(wind_df)

# Select a timestamp for interpolation (noon, when methane levels are typically higher)
noon_timestamp = pd.Timestamp('2025-02-10 12:00:00')
noon_methane = methane_gdf[methane_gdf['Timestamp'] == noon_timestamp]
noon_wind = wind_df_processed[wind_df_processed['Timestamp'] == noon_timestamp].iloc[0]

print(f"Loaded {len(methane_gdf)} methane records and {len(wind_df_processed)} wind records")
print(f"Found {len(noon_methane)} methane sensor readings for {noon_timestamp}")

## 2. Create Grid for Interpolation

In [ ]:
# Create a grid for interpolation
x_grid, y_grid, grid_points = create_grid(noon_methane, resolution=100, buffer=0.001)

# Display grid information
print(f"Created grid with dimensions {len(x_grid)}x{len(y_grid)}")
print(f"Grid bounds: X = [{x_grid.min():.6f}, {x_grid.max():.6f}], Y = [{y_grid.min():.6f}, {y_grid.max():.6f}]")

## 3. Inverse Distance Weighted (IDW) Interpolation

IDW is a deterministic method that estimates values at unknown points as weighted averages of known values. The weights are inversely proportional to the distance between the prediction location and the known values.

In [ ]:
# Perform IDW interpolation
xx_idw, yy_idw, zz_idw = idw_interpolation(methane_gdf, noon_timestamp, resolution=100, power=2)

# Plot the results
fig_idw = plot_interpolation(methane_gdf, xx_idw, yy_idw, zz_idw, noon_timestamp, 'IDW', add_basemap=True)

# Display the plot
plt.show()

## 4. Radial Basis Function (RBF) Interpolation

RBF is another deterministic method that estimates values at unknown points using a weighted sum of radial basis functions centered at each known point.

In [ ]:
# Perform RBF interpolation with multiquadric function
xx_rbf, yy_rbf, zz_rbf = rbf_interpolation(methane_gdf, noon_timestamp, resolution=100, function='multiquadric', epsilon=2)

# Plot the results
fig_rbf = plot_interpolation(methane_gdf, xx_rbf, yy_rbf, zz_rbf, noon_timestamp, 'RBF (Multiquadric)', add_basemap=True)

# Display the plot
plt.show()

## 5. Kriging Interpolation

Kriging is a geostatistical method that estimates values at unknown points based on the spatial autocorrelation of known values. It's particularly well-suited for environmental data.

In [ ]:
# Perform Kriging interpolation
xx_kriging, yy_kriging, zz_kriging = kriging_interpolation(methane_gdf, noon_timestamp, resolution=100, variogram_model='gaussian')

# Plot the results
fig_kriging = plot_interpolation(methane_gdf, xx_kriging, yy_kriging, zz_kriging, noon_timestamp, 'Ordinary Kriging', add_basemap=True)

# Display the plot
plt.show()

## 6. Compare Interpolation Methods

In [ ]:
# Create a grid of subplots to compare the methods
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Plot IDW interpolation
contour_idw = axes[0].contourf(xx_idw, yy_idw, zz_idw, cmap='YlOrRd', levels=15)
axes[0].scatter(noon_methane.geometry.x, noon_methane.geometry.y, c='black', s=30, edgecolor='white')
axes[0].set_title('IDW Interpolation')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')

# Plot RBF interpolation
contour_rbf = axes[1].contourf(xx_rbf, yy_rbf, zz_rbf, cmap='YlOrRd', levels=15)
axes[1].scatter(noon_methane.geometry.x, noon_methane.geometry.y, c='black', s=30, edgecolor='white')
axes[1].set_title('RBF Interpolation')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')

# Plot Kriging interpolation
contour_kriging = axes[2].contourf(xx_kriging, yy_kriging, zz_kriging, cmap='YlOrRd', levels=15)
axes[2].scatter(noon_methane.geometry.x, noon_methane.geometry.y, c='black', s=30, edgecolor='white')
axes[2].set_title('Kriging Interpolation')
axes[2].set_xlabel('Longitude')
axes[2].set_ylabel('Latitude')

# Add a colorbar
cbar = fig.colorbar(contour_kriging, ax=axes.ravel().tolist(), shrink=0.8)
cbar.set_label('Methane Concentration (ppm)')

# Adjust layout
plt.tight_layout()
plt.show()

## 7. Analyze Wind Influence on Interpolation

Let's create an interpolation that considers wind direction as a factor influencing methane dispersion.

In [ ]:
def wind_adjusted_idw(methane_gdf, wind_df, timestamp, resolution=100, power=2, wind_factor=0.3):
    """
    IDW interpolation with wind direction adjustment
    """
    # Filter data for the given timestamp
    time_gdf = methane_gdf[methane_gdf['Timestamp'] == pd.to_datetime(timestamp)].copy()
    time_wind = wind_df[wind_df['Timestamp'] == pd.to_datetime(timestamp)].iloc[0]
    
    # Extract points and values
    points = np.array([(point.x, point.y) for point in time_gdf.geometry])
    values = time_gdf['Methane_Concentration (ppm)'].values
    
    # Get wind vector components
    u = time_wind['U']
    v = time_wind['V']
    
    # Create the grid
    x_grid, y_grid, grid_points = create_grid(time_gdf, resolution)
    xx, yy = np.meshgrid(x_grid, y_grid)
    
    # Calculate IDW with wind adjustment
    zz = np.zeros(xx.shape)
    for i in range(len(xx)):
        for j in range(len(xx[0])):
            # Calculate distances from grid point to all data points
            dx = xx[i,j] - points[:,0]
            dy = yy[i,j] - points[:,1]
            
            # Adjust distances based on wind direction (points downwind get more influence)
            wind_alignment = (dx*u + dy*v) / (np.sqrt(dx**2 + dy**2) * np.sqrt(u**2 + v**2) + 1e-10)
            wind_adjustment = 1.0 - wind_factor * (wind_alignment + 1) / 2  # Scale to [1-wind_factor, 1]
            
            # Calculate adjusted distances
            distances = np.sqrt(dx**2 + dy**2) * wind_adjustment
            
            # Avoid division by zero
            distances[distances < 1e-10] = 1e-10
            
            # Calculate weights
            weights = 1.0 / (distances ** power)
            
            # Calculate weighted average
            zz[i,j] = np.sum(weights * values) / np.sum(weights)
    
    return xx, yy, zz

# Execute wind-adjusted IDW interpolation
xx_wind_idw, yy_wind_idw, zz_wind_idw = wind_adjusted_idw(
    methane_gdf, wind_df_processed, noon_timestamp, resolution=100, wind_factor=0.3
)

# Plot the results
fig, ax = plt.subplots(figsize=(12, 10))

# Plot interpolated surface
contour = ax.contourf(xx_wind_idw, yy_wind_idw, zz_wind_idw, cmap='YlOrRd', levels=15)

# Add sensor locations
ax.scatter(
    noon_methane.geometry.x, 
    noon_methane.geometry.y, 
    c=noon_methane['Methane_Concentration (ppm)'],
    cmap='YlOrRd',
    edgecolor='k',
    s=80
)

# Add wind vector
scale = 0.002  # Scale factor for arrow length
u = noon_wind['U']
v = noon_wind['V']

# Center point for wind vector
center_x = noon_methane.geometry.x.min() + (noon_methane.geometry.x.max() - noon_methane.geometry.x.min()) * 0.2
center_y = noon_methane.geometry.y.min() + (noon_methane.geometry.y.max() - noon_methane.geometry.y.min()) * 0.2

ax.arrow(center_x, center_y, u*scale, v*scale, 
         head_width=0.0001, head_length=0.0002, 
         fc='blue', ec='blue', width=0.00005)

# Add wind legend
ax.text(center_x, center_y - 0.0005, 
        f"Wind: {noon_wind['Wind_Speed (m/s)']:.1f} m/s, {noon_wind['Wind_Direction (°)']:.0f}°", 
        color='blue', fontsize=12, ha='center')

# Add sensor labels
for idx, row in noon_methane.iterrows():
    ax.annotate(
        row['Sensor_ID'], 
        (row.geometry.x, row.geometry.y),
        xytext=(5, 5),
        textcoords="offset points",
        fontsize=10,
        color='black',
        fontweight='bold'
    )

# Add colorbar
cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('Methane Concentration (ppm)', fontsize=12)

# Set title and axis labels
ax.set_title(f'Wind-Adjusted IDW Interpolation of Methane\n{noon_timestamp}', fontsize=14)
ax.set_xlabel('Longitude', fontsize=12)
ax.set_ylabel('Latitude', fontsize=12)

# Add basemap if available
try:
    ctx.add_basemap(ax, crs=noon_methane.crs.to_string(), source=ctx.providers.OpenStreetMap.Mapnik)
except Exception as e:
    print(f"Could not add basemap: {e}")

# Adjust layout
plt.tight_layout()
plt.show()

## 8. Compare Regular IDW and Wind-Adjusted IDW

Let's directly compare the standard IDW interpolation with our wind-adjusted version to see how wind direction affects methane dispersion predictions.

In [ ]:
# Create side-by-side comparison
fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# Plot standard IDW
contour_std = axes[0].contourf(xx_idw, yy_idw, zz_idw, cmap='YlOrRd', levels=15)
axes[0].scatter(noon_methane.geometry.x, noon_methane.geometry.y, c='black', s=30, edgecolor='white')
axes[0].set_title('Standard IDW Interpolation')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')

# Plot wind-adjusted IDW
contour_wind = axes[1].contourf(xx_wind_idw, yy_wind_idw, zz_wind_idw, cmap='YlOrRd', levels=15)
axes[1].scatter(noon_methane.geometry.x, noon_methane.geometry.y, c='black', s=30, edgecolor='white')

# Add wind vector to second plot
center_x = noon_methane.geometry.x.min() + (noon_methane.geometry.x.max() - noon_methane.geometry.x.min()) * 0.2
center_y = noon_methane.geometry.y.min() + (noon_methane.geometry.y.max() - noon_methane.geometry.y.min()) * 0.2

axes[1].arrow(center_x, center_y, noon_wind['U']*0.002, noon_wind['V']*0.002, 
              head_width=0.0001, head_length=0.0002, 
              fc='blue', ec='blue', width=0.00005)
axes[1].set_title('Wind-Adjusted IDW Interpolation')
axes[1].set_xlabel('Longitude')

# Add common colorbar
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(contour_std, cax=cbar_ax)
cbar.set_label('Methane Concentration (ppm)')

plt.suptitle('Comparison of Interpolation Methods', fontsize=16, y=0.98)
plt.tight_layout(rect=[0, 0, 0.9, 0.95])
plt.show()

## 9. Time Series of Interpolated Methane Concentrations

Now, let's create a series of interpolations throughout the day to see how methane concentrations evolve over time.

In [ ]:
# Select key timestamps throughout the day
key_timestamps = [
    pd.Timestamp('2025-02-10 06:00:00'),  # Morning
    pd.Timestamp('2025-02-10 12:00:00'),  # Noon
    pd.Timestamp('2025-02-10 18:00:00'),  # Evening
    pd.Timestamp('2025-02-10 22:00:00')   # Night
]

# Create a grid of subplots for the time series
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, timestamp in enumerate(key_timestamps):
    # Get wind data for this timestamp
    ts_wind = wind_df_processed[wind_df_processed['Timestamp'] == timestamp].iloc[0]
    
    # Perform kriging interpolation
    try:
        xx_ts, yy_ts, zz_ts = kriging_interpolation(methane_gdf, timestamp, resolution=80)
        
        # Plot interpolated surface
        contour = axes[i].contourf(xx_ts, yy_ts, zz_ts, cmap='YlOrRd', levels=15)
        
        # Add sensor locations
        ts_gdf = methane_gdf[methane_gdf['Timestamp'] == timestamp]
        axes[i].scatter(ts_gdf.geometry.x, ts_gdf.geometry.y, c='black', s=30, edgecolor='white')
        
        # Add wind vector
        u, v = ts_wind['U'], ts_wind['V']
        center_x = ts_gdf.geometry.x.mean()
        center_y = ts_gdf.geometry.y.mean() - 0.0005
        
        axes[i].arrow(center_x, center_y, u*0.002, v*0.002, 
                     head_width=0.0001, head_length=0.0002, 
                     fc='blue', ec='blue', width=0.00005)
        
        axes[i].text(center_x, center_y - 0.0008, 
                    f"Wind: {ts_wind['Wind_Speed (m/s)']:.1f} m/s, {ts_wind['Wind_Direction (°)']:.0f}°", 
                    color='blue', fontsize=10, ha='center')
        
        axes[i].set_title(f'Time: {timestamp.strftime("%Y-%m-%d %H:%M")}', fontsize=12)
        axes[i].set_xlabel('Longitude')
        axes[i].set_ylabel('Latitude')
        
    except Exception as e:
        print(f"Error processing timestamp {timestamp}: {e}")
        axes[i].text(0.5, 0.5, f"Error: {e}", ha='center', transform=axes[i].transAxes)

# Add common colorbar
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
cbar = fig.colorbar(contour, cax=cbar_ax)
cbar.set_label('Methane Concentration (ppm)')

plt.suptitle('Methane Concentration Throughout the Day (Kriging Interpolation)', fontsize=16, y=0.98)
plt.tight_layout(rect=[0, 0, 0.9, 0.95])
plt.show()

## 10. Conclusions from Interpolation Analysis

Based on our interpolation analysis, we can draw the following conclusions:

1. **Spatial Distribution of Methane**: 
   - The interpolation reveals clear spatial patterns in methane concentrations.
   - Higher concentrations appear to be clustered in specific areas, which may indicate emission sources.

2. **Method Comparison**:
   - **IDW**: Produces smoother transitions but may not capture local variations as well.
   - **RBF**: Generally creates more smoothed surfaces which might miss some localized patterns.
   - **Kriging**: Often provides the most statistically robust estimates, accounting for spatial autocorrelation.
   - **Wind-adjusted IDW**: Incorporates directional effects, showing how pollutants might disperse downwind.

3. **Wind Influence**:
   - The wind-adjusted interpolation demonstrates how wind patterns affect methane dispersion.
   - Areas downwind from high-concentration zones show elevated methane levels in the model.

4. **Temporal Patterns**:
   - The time series analysis shows how methane concentrations vary throughout the day.
   - Peak concentrations appear around noon, which may correlate with industrial activities or atmospheric conditions.

## 11. Next Steps

1. Perform spatial clustering to identify high-risk zones.
2. Develop predictive models that can forecast methane concentrations based on wind patterns.
3. Integrate interpolation results with other environmental factors to enhance understanding of dispersion dynamics.